This file will test which comnination of GLCM features provide the highest accuracy

In [ ]:
import math 
import os
import json
import csv
import cv2 as cv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from skimage.feature import graycoprops, graycomatrix, local_binary_pattern, hog
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from skimage import data, exposure, io
from sklearn.preprocessing import StandardScaler
from itertools import combinations

In [ ]:
#updated json file path
updated_json_file = r"C:\Users\mdrsp\Desktop\School\For thesis\spv_images\InfraredSolarModules\module_metadata_update.json"

# images folder path
path_2_images = r"C:\Users\mdrsp\Desktop\School\For thesis\spv_images\InfraredSolarModules"

# path to any image
sample_img = r"C:\Users\mdrsp\Desktop\School\For thesis\spv_images\InfraredSolarModules\images\0.jpg"

def path2img(path):
    img = io.imread(path,as_gray=True)
    return img

In [39]:
def glcm_features(img,features=['contrast', 'dissimilarity', 'homogeneity', 'ASM', 'correlation', 'energy']):    
    # Calculate the GLCM
    glcm = graycomatrix(img, distances=[1,2,3], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256, symmetric=True, normed=True)
    feature_vector = []
    for feature in features:
        feature_value = graycoprops(glcm, feature)
        feature_vector.append(feature_value)
    return feature_vector


In [ ]:
print(glcm_features(path2img(sample_img),['contrast','dissimilarity','homogeneity']))

In [47]:
# save teh GLCM values of the 20k images into arrays
file = open(updated_json_file)
data = json.load(file)

anomaly_list = []
feature_list = []

os.chdir(path_2_images)
for key,value in data.items():
    current_img_path = value['image_filepath']
    anomaly = value['anomaly_class']

    img = path2img(current_img_path)
     
    feature_list.append(glcm_features(img))
    anomaly_list.append(anomaly) 

In [54]:
def get_features(values,features= ['contrast', 'dissimilarity', 'homogeneity', 'ASM', 'correlation', 'energy']):
    new_values = []
    for row in values:
        temp = []
        temp.append(row[0]) if 'contrast' in features else None
        temp.append(row[1]) if 'dissimilarity' in features else None
        temp.append(row[2]) if 'homogeneity' in features else None
        temp.append(row[3]) if 'ASM' in features else None
        temp.append(row[4]) if 'correlation' in features else None
        temp.append(row[5]) if 'energy' in features else None
        temp = np.ravel(temp)
        new_values.append(temp)
    return np.array(new_values)

In [58]:
feature_vector = get_features(feature_list) # the list of glcm values to be used for training

images = np.array(feature_vector)
labels = np.array(anomaly_list)

In [59]:
print(images[0])

[4.71097826e+01 8.17179487e+01 4.38974359e+01 8.14894091e+01
 1.55646591e+02 8.17179487e+01 1.16151316e+02 8.14894091e+01
 2.68089286e+02 2.31035885e+02 1.81272523e+02 2.30721292e+02
 4.61847826e+00 6.57859532e+00 4.69230769e+00 6.32998885e+00
 8.30113636e+00 6.57859532e+00 8.10745614e+00 6.32998885e+00
 1.06988095e+01 1.12559809e+01 1.05990991e+01 1.09007177e+01
 2.39705537e-01 1.47954025e-01 2.03650099e-01 1.62216918e-01
 1.39733351e-01 1.47954025e-01 1.21477118e-01 1.62216918e-01
 1.28447440e-01 9.73752777e-02 8.50137335e-02 1.02454777e-01
 1.63220699e-03 1.32735279e-03 1.55747772e-03 1.32300285e-03
 1.30488120e-03 1.32735279e-03 1.15239785e-03 1.32300285e-03
 1.29109977e-03 1.08099059e-03 1.08617908e-03 1.15396282e-03
 9.53694801e-01 9.15020146e-01 9.57824915e-01 9.15239210e-01
 8.38955348e-01 9.15020146e-01 8.83696362e-01 9.15239210e-01
 7.20720397e-01 7.32336679e-01 8.12544451e-01 7.33302142e-01
 4.04005816e-02 3.64328532e-02 3.94648922e-02 3.63731061e-02
 3.61231394e-02 3.643285

In [57]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
clf = SVC(kernel='poly', C=1, random_state=42)
clf.fit(X_train, y_train)
accuracy = clf.score(X_test, y_test)
print(f'Accuracy: {accuracy:.3f}')

Accuracy: 0.531
